In [13]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Unclassified Splits/Unattributable Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")

In [14]:
excel_path_unattributable = path_equity_precision_llm_folder + "//llm training/Unclassified Splits/Unattributable Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")

In [16]:
input_unattributable = clean_input(input_path = excel_path_unattributable)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM//llm training/Unclassified Splits/Unattributable Part '

In [10]:
import pandas as pd
import re

n_json_splits = 11
# Concatenate multiple CSVs
results = None
for scenario in range(1, n_json_splits + 1):
    file_path = f"{path_equity_precision_llm_folder}/llm training/Unclassified Splits/Unattributable Part {scenario}_results.csv"
    temp_df = pd.read_csv(file_path)

    if results is None:
        results = temp_df
    else:
        results = pd.concat([results, temp_df], ignore_index=True)

# Merge datasets
merged_df_unattributable = input_unattributable.merge(results, left_on='PMID', right_on='pmid', how='left')

# Standardize source population categories: now loaded from execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
# def standardize_population(value):
#     if pd.isna(value) or value.strip().lower() in ["unknown", "not applicable", "n/a", "N/A", "unk", "not specified", "Unknown", "Not Applicable", "Not Specified"]:
#         return "Unknown"
#     return value.strip()

merged_df_unattributable['orig_source_population'] = merged_df_unattributable['orig_source_population'].apply(standardize_population)
merged_df_unattributable['gpt_source_population'] = merged_df_unattributable['gpt_source_population'].apply(standardize_population)

# Function for classification comparison: now loaded from execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")
# def adjusted_source_population_match(row):
#     """
#     Adjusted logic for source population matching:
#     - If either column is NaN or categorized as "Unknown", classify it as 'Unknown'.
#     - Match is True if GPT's classification contains at least one exact category from the original classification.
#     """
#     orig_population = row['orig_source_population']
#     gpt_population = row['gpt_source_population']
    
#     orig_set = set(orig_population.split(', '))
#     gpt_set = set(gpt_population.split(', '))

#     # If GPT predicted 'Unknown', it doesn't match any known category.
#     if gpt_population == 'Unknown':
#         return orig_population == 'Unknown'  # Only match if both are 'Unknown'

#     # Match only if GPT contains at least one exact category from the original classification
#     return any(category in orig_set for category in gpt_set)


merged_df_unattributable['source_population_match'] = merged_df_unattributable.apply(adjusted_source_population_match, axis=1)

# Crosstab summary for precision medicine
summary_precision_medicine = crosstab_summary(merged_df_unattributable, truth='orig_precision_medicine', test='gpt_precision_medicine')
summary_diabetes = crosstab_summary(merged_df_unattributable, truth='orig_diabetes', test='gpt_diabetes')
summary_primary_study = crosstab_summary(merged_df_unattributable, truth='orig_primary_study', test='gpt_primary_study')

summary_precision_medicine['variable'] = 'Precision Medicine'
summary_diabetes['variable'] = 'Diabetes'
summary_primary_study['variable'] = 'Primary Study'

# Generate updated crosstab for source population
t_source_population = pd.crosstab(merged_df_unattributable['source_population_match'], merged_df_unattributable['orig_source_population'])

prop_correct_source_population = t_source_population.loc[True].sum() / t_source_population.sum().sum()
prop_correct_source_population

summary_source_population = pd.DataFrame({'variable': 'Source Population', 'Accuracy': prop_correct_source_population}, index=[0])

df_summary = pd.concat([summary_precision_medicine, summary_diabetes, summary_primary_study])

df_summary = pd.concat([df_summary,summary_source_population])

df_summary.to_csv(path_equity_precision_llm_repo + '/preprocessing/epl04_combined output_Unattributable.csv', index=False)

NameError: name 'input_unattributable' is not defined